## Store the data into neo4j graph database

1. Find all the distinct league nodes and create them in neo4j database
2. Find all the distinct game nodes and create them in neo4j database
3. Find all the distinct team nodes and create them
4. Find all the distinct player nodes and create them

- Once this is done, we can make relationships between them by passing through the whole data. 

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")
%matplotlib inline

In [2]:
pd.set_option("display.max_columns", None)

In [3]:
# match_data = get_match_data()
match_data = pd.read_csv("csv_files/Match Data 20220915.csv")

In [4]:
match_data.head(30)

,gameid,datacompleteness,url,league,year,split,playoffs,date,game,patch,participantid,side,position,playername,playerid,teamname,teamid,champion,ban1,ban2,ban3,ban4,ban5,gamelength,result,kills,deaths,assists,teamkills,teamdeaths,doublekills,triplekills,quadrakills,pentakills,firstblood,firstbloodkill,firstbloodassist,firstbloodvictim,team kpm,ckpm,firstdragon,dragons,opp_dragons,elementaldrakes,opp_elementaldrakes,infernals,mountains,clouds,oceans,chemtechs,hextechs,dragons (type unknown),elders,opp_elders,firstherald,heralds,opp_heralds,firstbaron,barons,opp_barons,firsttower,towers,opp_towers,firstmidtower,firsttothreetowers,turretplates,opp_turretplates,inhibitors,opp_inhibitors,damagetochampions,dpm,damageshare,damagetakenperminute,damagemitigatedperminute,wardsplaced,wpm,wardskilled,wcpm,controlwardsbought,visionscore,vspm,totalgold,earnedgold,earned gpm,earnedgoldshare,goldspent,gspd,total cs,minionkills,monsterkills,monsterkillsownjungle,monsterkillsenemyjungle,cspm,goldat10,xpat10,csat10,opp_goldat10,opp_xpat10,opp_csat10,golddiffat10,xpdiffat10,csdiffat10,killsat10,assistsat10,deathsat10,opp_killsat10,opp_assistsat10,opp_deathsat10,goldat15,xpat15,csat15,opp_goldat15,opp_xpat15,opp_csat15,golddiffat15,xpdiffat15,csdiffat15,killsat15,assistsat15,deathsat15,opp_killsat15,opp_assistsat15,opp_deathsat15
0,ESPORTSTMNT01_2690210,complete,NaN,LCK CL,2022,Spring,0,2022-01-10 07:44:08,1,12.01,1,Blue,top,Soboro,oe:player:38e0af7278d6769d0c81d7c4b47ac1e,Fredit BRION Challengers,oe:team:68911b3329146587617ab2973106e23,Renekton,Karma,Caitlyn,Syndra,Thresh,Lulu,1713,0,2,3,2,9,19,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3152,0.9807,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,15768.0,552.2942,0.278784,1072.3993,777.7933,8.0,0.2802,6.0,0.2102,5.0,26.0,0.9107,10934,7164.0,250.9282,0.253859,10275.0,NaN,231.0,220.0,11.0,NaN,NaN,8.0911,3228.0,4909.0,89.0,3176.0,4953.0,81.0,52.0,-44.0,8.0,0.0,0.0,0.0,0.0,0.0,0.0,5025.0,7560.0,135.0,4634.0,7215.0,121.0,391.0,345.0,14.0,0.0,1.0,0.0,0.0,1.0,0.0
1,ESPORTSTMNT01_2690210,complete,NaN,LCK CL,2022,Spring,0,2022-01-10 07:44:08,1,12.01,2,Blue,jng,Raptor,oe:player:637ed20b1e41be1c51bd1a4cb211357,Fredit BRION Challengers,oe:team:68911b3329146587617ab2973106e23,Xin Zhao,Karma,Caitlyn,Syndra,Thresh,Lulu,1713,0,2,5,6,9,19,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.3152,0.9807,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,1.0,11765.0,412.0841,0.208009,944.2732,650.1576,6.0,0.2102,18.0,0.6305,6.0,48.0,1.6813,9138,5368.0,188.0210,0.190220,8750.0,NaN,148.0,33.0,115.0,NaN,NaN,5.1839,3429.0,3484.0,58.0,2944.0,3052.0,63.0,485.0,432.0,-5.0,1.0,2.0,0.0,0.0,0.0,1.0,5366.0,5320.0,89.0,4825.0,5595.0,100.0,541.0,-275.0,-11.0,2.0,3.0,2.0,0.0,5.0,1.0
2,ESPORTSTMNT01_2690210,complete,NaN,LCK CL,2022,Spring,0,2022-01-10 07:44:08,1,12.01,3,Blue,mid,Feisty,oe:player:d1ae0e2f9f3ac1e0e0cdcb86504ca77,Fredit BRION Challengers,oe:team:68911b3329146587617ab2973106e23,LeBlanc,Karma,Caitlyn,Syndra,Thresh,Lulu,1713,0,2,2,3,9,19,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3152,0.9807,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,14258.0,499.4046,0.252086,581.6462,227.7758,19.0,0.6655,7.0,0.2452,7.0,29.0,1.0158,9715,5945.0,208.2312,0.210665,8725.0,NaN,193.0,177.0,16.0,NaN,NaN,6.7601,3283.0,4556.0,81.0,3121.0,4485.0,81.0,162.0,71.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,5118.0,6942.0,120.0,5593.0,6789.0,119.0,-475.0,153.0,1.0,0.0,3.0,0.0,3.0,3.0,2.0
3,ESPORTSTMNT01_2690210,complete,NaN,LCK CL,2022,Spring,0,2022-01-10 07:44:08,1,12.01,4,Blue,bot,Gamin,oe:player:998b3e49b01ecc41eacc392477a98cf,Fredit BRION Challengers,oe:team:68911b3329146587617ab2973106e23,Samira,Karma,Caitlyn,Syndra,Thresh,Lulu,1713,0,2,4,2,9,19,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.3152,0.9807,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.

## Removing invalid game ids

In [5]:
invalid_game_ids = []

# removing the invalid game data for invalid game ids
for gid in match_data.gameid.values:
    if not isinstance(gid, float):
        if "NA" in gid:
            invalid_game_ids.append(gid)

match_data = match_data[~match_data["gameid"].isin(invalid_game_ids)].copy()

## League Nodes

In [6]:
# lp = pd.DataFrame(match_data.league.unique())
# lp.columns = ["league_name"]
league_names_list = match_data.league.unique().tolist()
len(league_names_list)

44

In [7]:
# match_data.league.value_counts(), len(match_data.league.value_counts())

In [8]:
# league_data = pd.DataFrame(match_data.league.value_counts())
# league_data = league_data.reset_index()
# league_data.columns = ["league_name","count"]

In [9]:
# league_data.league_name

In [10]:
# for name in league_data.league_name:
#     print(name)
#     break

## Make nodes in the neo4j database

In [11]:
transactions = []

for name in league_names_list:
    q = f"create (l:League {{name: '{name}' }})"
    transactions.append(q)

In [12]:
transactions

["create (l:League {name: 'LCK CL' })",
 "create (l:League {name: 'LPL' })",
 "create (l:League {name: 'NLC' })",
 "create (l:League {name: 'SL' })",
 "create (l:League {name: 'UL' })",
 "create (l:League {name: 'PRM' })",
 "create (l:League {name: 'LCK' })",
 "create (l:League {name: 'LFL' })",
 "create (l:League {name: 'Proving Grounds Circuit' })",
 "create (l:League {name: 'LEC' })",
 "create (l:League {name: 'LCS' })",
 "create (l:League {name: 'LFL2' })",
 "create (l:League {name: 'GLL' })",
 "create (l:League {name: 'HM' })",
 "create (l:League {name: 'ESLOL' })",
 "create (l:League {name: 'EBL' })",
 "create (l:League {name: 'LPLOL' })",
 "create (l:League {name: 'PGN' })",
 "create (l:League {name: 'LCSA' })",
 "create (l:League {name: 'LVP DDH' })",
 "create (l:League {name: 'TRA' })",
 "create (l:League {name: 'TCL' })",
 "create (l:League {name: 'CBLOL' })",
 "create (l:League {name: 'LCO' })",
 "create (l:League {name: 'LHE' })",
 "create (l:League {name: 'GL' })",
 "creat

In [13]:
from neo4j import GraphDatabase

def execute_commands(transactions):
    database_connection = GraphDatabase.driver(uri="bolt://localhost:7687", auth=("neo4j", "password"))
    session = database_connection.session()
    for t in transactions:
        session.run(t)

In [14]:
execute_commands(transactions)

## Game nodes

In [15]:
games_list = match_data.gameid.unique().tolist()

In [16]:
len(games_list)

10582

In [19]:
transactions = []
for g in games_list:
    q = f"create (g:Game {{gameid:'{g}'}})"
    transactions.append(q)

In [20]:
execute_commands(transactions)

## Team Nodes 

In [25]:
team_data = match_data[match_data["position"] == "team"].teamname.unique().tolist()
len(team_data)

471

In [28]:
transactions = []
for t in team_data:
    q = f"""create (t:Team {{name:"{t}"}})"""
    transactions.append(q)

In [29]:
transactions

['create (t:Team {name:"Fredit BRION Challengers"})',
 'create (t:Team {name:"Nongshim RedForce Challengers"})',
 'create (t:Team {name:"T1 Challengers"})',
 'create (t:Team {name:"Liiv SANDBOX Challengers"})',
 'create (t:Team {name:"Oh My God"})',
 'create (t:Team {name:"ThunderTalk Gaming"})',
 'create (t:Team {name:"KT Rolster Challengers"})',
 'create (t:Team {name:"Gen.G Challengers"})',
 'create (t:Team {name:"DWG KIA Challengers"})',
 'create (t:Team {name:"DRX Challengers"})',
 'create (t:Team {name:"FunPlus Phoenix"})',
 'create (t:Team {name:"Royal Never Give Up"})',
 'create (t:Team {name:"Kwangdong Freecs Challengers"})',
 'create (t:Team {name:"Hanwha Life Esports Challengers"})',
 'create (t:Team {name:"Vanir"})',
 'create (t:Team {name:"Team Singularity"})',
 'create (t:Team {name:"Movistar Riders"})',
 'create (t:Team {name:"Team Heretics"})',
 'create (t:Team {name:"BISONS ECLUB"})',
 'create (t:Team {name:"G2 Arctic"})',
 'create (t:Team {name:"Bifrost"})',
 'create 

In [30]:
execute_commands(transactions)

## PLayer Nodes

In [39]:
# player_data = match_data[match_data["position"] != "team"]

In [63]:
# format_dict = {
#     "": np.nan, 
#     "nan": np.nan, 
#     "null":np.nan
# }

# player_data["playerid"] = player_data["playerid"].str.strip()
# match_data = player_data.replace({
#     "playerid": format_dict,
# })

In [64]:
# if "null" in player_data["playerid"].str.strip():
#     print("yes")

In [65]:
# player_data = player_data[["playername", "playerid"]].drop_duplicates()

In [72]:
players_list = match_data.playername.unique().tolist()
len(players_list)

3165

In [73]:
transactions = []
for p in players_list:
    q = f"""
    create (n:Player {{name: "{p}"}})
    """
    transactions.append(q)

In [74]:
execute_commands(transactions)